In [91]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV, SelectKBest, f_classif

from sklearn.tree import DecisionTreeClassifier

from hyperopt import hp, fmin, tpe

In [98]:
data1 = pd.read_csv('mushrooms.csv')
data1.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [99]:

data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
class                       8124 non-null object
cap-shape                   8124 non-null object
cap-surface                 8124 non-null object
cap-color                   8124 non-null object
bruises                     8124 non-null object
odor                        8124 non-null object
gill-attachment             8124 non-null object
gill-spacing                8124 non-null object
gill-size                   8124 non-null object
gill-color                  8124 non-null object
stalk-shape                 8124 non-null object
stalk-root                  8124 non-null object
stalk-surface-above-ring    8124 non-null object
stalk-surface-below-ring    8124 non-null object
stalk-color-above-ring      8124 non-null object
stalk-color-below-ring      8124 non-null object
veil-type                   8124 non-null object
veil-color                  8124 non-null object
ring-number

In [105]:
x = pd.get_dummies(data1.iloc[:, 1:120])
y = data.iloc[:, :1]
x.head()

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [106]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 100)
xtest.head()
#xtrain.columns

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
2690,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
5021,0,0,0,0,0,1,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
2708,0,0,0,0,0,1,0,0,0,1,...,0,0,1,1,0,0,0,0,0,0
1405,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2935,0,0,1,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0


In [107]:
# scaler = StandardScaler()
# xtrain[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
#        'exang', 'oldpeak', 'slope', 'ca', 'thal']] = scaler.fit_transform(xtrain[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
#        'exang', 'oldpeak', 'slope', 'ca', 'thal']])

# xtest[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
#        'exang', 'oldpeak', 'slope', 'ca', 'thal']] = scaler.fit_transform(xtest[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
#        'exang', 'oldpeak', 'slope', 'ca', 'thal']])

In [108]:
xtrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6499 entries, 134 to 5640
Columns: 117 entries, cap-shape_b to habitat_w
dtypes: uint8(117)
memory usage: 793.3 KB


In [109]:
skb = SelectKBest(k=10)
skb.fit(xtrain, ytrain)
skb.get_support()

C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [82] are constant.
  UserWarning)
C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True, False, False,  True, False, False,
        True, False, False, False, False, False, False, False,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [112]:
estimator = DecisionTreeClassifier()
rfecv = RFECV(estimator)
rfecv.fit(xtrain, ytrain)
col = rfecv.get_support()

C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [118]:
features = xtrain.columns
new_features = []
for bool, feature in zip(col, features):
    if bool:
        new_features.append(feature)

new_features

['bruises_f',
 'odor_a',
 'odor_l',
 'odor_n',
 'gill-size_n',
 'stalk-root_c',
 'stalk-surface-below-ring_y',
 'spore-print-color_r']

In [119]:
train2 = pd.DataFrame(xtrain, columns=new_features)
train2.head()

,bruises_f,odor_a,odor_l,odor_n,gill-size_n,stalk-root_c,stalk-surface-below-ring_y,spore-print-color_r
134,0,1,0,0,1,0,0,0
2348,1,0,0,1,0,0,0,0
5515,1,0,0,1,1,0,0,0
4897,1,0,0,0,0,0,0,0
3816,0,0,0,1,0,0,0,0


In [121]:
# WIth Bayesian Optimization and KNN
def objective_func(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
 
    #clf.fit(x_pca,y_train.values.ravel())    
    #y_pred = clf.predict(x_valpca)
    #f1 = -(f1_score(y_pred,y_val,  average='macro'))
    f1 = -(cross_val_score(clf, train2,ytrain.values.ravel(), cv=3, scoring = 'f1_macro').mean())
    return f1
space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,4000, step =1)),
        'metric':hp.choice('metric', ["euclidean","manhattan"])}

best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=100)
print(best_classifier)

100%|██████████| 100/100 [07:32<00:00,  4.66s/it, best loss: -0.9853644010075109]
{'metric': 0, 'n_neighbors': 53}
